In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data-p3/joined_all.csv')
df.head()


,Pass_ID,Sch_Departure,Act_Departure,C0 - S2,C_Start,C_Avg,S2,Wait_Time,Day_of_Week,Period_of_Week,Airfield,Season
0,5368296,2028-09-03 13:12,2028-09-03 13:12,1,1,1.0,2028-09-03 11:26,3.0,7 - SUN,2 - WEEKEND,AUC,3 - SUMMER
1,5480057,2028-09-15 9:36,2028-09-15 9:36,1,1,1.0,2028-09-15 7:45,3.0,5 - FRI,1 - WEEKDAY,AUC,3 - SUMMER
2,5349077,2028-09-01 10:21,2028-09-01 11:09,1,1,1.0,2028-09-01 7:58,5.0,5 - FRI,1 - WEEKDAY,AUC,3 - SUMMER
3,5349207,2028-09-01 10:21,2028-09-01 11:09,1,1,1.0,2028-09-01 8:04,4.0,5 - FRI,1 - WEEKDAY,AUC,3 - SUMMER
4,5349308,2028-09-01 10:21,2028-09-01 11:09,1,1,1.0,2028-09-01 8:13,7.0,5 - FRI,1 - WEEKDAY,AUC,3 - SUMMER


In [ ]:
df['S2'] = pd.to_datetime(df['S2'])
# df['wait_time_delta'] = pd.to_timedelta(df['Wait_Time'], unit='m')
# df['S1'] =df['S2'] - df['Wait_Time']

UFuncTypeError: ufunc 'subtract' cannot use operands with types dtype('<M8[ns]') and dtype('float64')

In [4]:
def determine_cluster(timestamp):

    is_weekend = timestamp.weekday() >= 5  
    hour = timestamp.hour
    
    if 0 <= hour < 4:
        cluster = "0:00 - 4:00"
    elif 4 <= hour < 8:
        cluster = "4:00 - 8:00"
    elif 8 <= hour < 12:
        cluster = "8:00 - 12:00"
    elif 12 <= hour < 16:
        cluster = "12:00 - 16:00"
    elif 16 <= hour < 20:
        cluster = "16:00 - 20:00"
    else:
        cluster = "20:00 - 0:00"
    
    return f"{'Weekend' if is_weekend else 'Weekday'} - {cluster}"

df['cluster'] = df['S2'].apply(determine_cluster)

In [34]:
def impute_exponential_by_cluster(df, cluster_col='cluster', target_col='Wait_Time', time_col='S2'):
    df_imputed = df.copy()
    
    # Calculate means for each cluster
    cluster_means = df.groupby(cluster_col)[target_col].mean()
    
    # Function to find an existing wait time or sample from exponential
    def impute_value(row):
        if pd.isna(row[target_col]):  # If value is missing
            cluster = row[cluster_col]
            arrival_time = row[time_col]
            
            # Check if someone else in the same cluster and arrival time has a Wait_Time
            same_time_value = df_imputed[
                (df_imputed[cluster_col] == cluster) &
                (df_imputed[time_col] == arrival_time) &
                (~pd.isna(df_imputed[target_col]))
            ][target_col]
            
            if not same_time_value.empty:  # If a valid Wait_Time exists, use it
                return same_time_value.iloc[0]
            
            # Otherwise, sample from exponential
            scale = cluster_means[cluster]  # Mean of the feature for the cluster
            if scale > 0:  # Ensure the scale is valid
                return np.random.exponential(scale=scale)
        
        # Return original value if not missing
        return row[target_col]
    
    # Apply the imputation
    df_imputed['wait_time_imp'] = df_imputed.apply(impute_value, axis=1)
    return df_imputed


In [35]:
df = impute_exponential_by_cluster(df)

/var/folders/8t/19klq0xd5w7cf39d3893ts0w0000gn/T/ipykernel_38055/2439256362.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cluster_means = df.groupby(cluster_col)[target_col].mean()


In [33]:
cluster_ord = [
    "Weekday - 0:00 - 4:00", "Weekday - 4:00 - 8:00", "Weekday - 8:00 - 12:00", 
    "Weekday - 12:00 - 16:00", "Weekday - 16:00 - 20:00", "Weekday - 20:00 - 0:00",
    "Weekend - 0:00 - 4:00", "Weekend - 4:00 - 8:00", "Weekend - 8:00 - 12:00", 
    "Weekend - 12:00 - 16:00", "Weekend - 16:00 - 20:00", "Weekend - 20:00 - 0:00"
]

df['cluster'] = pd.Categorical(df['cluster'], categories=cluster_ord, ordered=True)
df = df.sort_values('cluster')

In [36]:
df["S1_imp"] = df['S2'] - pd.to_timedelta(df['wait_time_imp'])
# df["cluster_S1"] = df['S1_imp'].apply(determine_cluster)

In [97]:
def cluster_hours(timestamp):
    is_weekend = timestamp.weekday() >= 5

    # subtracting 18 missing weekdays and 6 missing weekends, see appendix
    weekday = (153 - 44) * 4 - (18 * 4)
    weekend = 44 * 4 - (6 * 4)

    
    return weekend if is_weekend else weekday

df['cluster_hours'] = df['S2'].apply(cluster_hours)

In [98]:
df[['cluster','cluster_hours']]

,cluster,cluster_hours
57071,Weekday - 4:00 - 8:00,364
56736,Weekday - 4:00 - 8:00,364
56735,Weekday - 4:00 - 8:00,364
91991,Weekday - 4:00 - 8:00,364
91992,Weekday - 4:00 - 8:00,364
...,...,...
15664,Weekend - 20:00 - 0:00,152
15665,Weekend - 20:00 - 0:00,152
15666,Weekend - 20:00 - 0:00,152
15661,Weekend - 20:00 - 0:00,152


In [99]:
arrival_counts = df.groupby('cluster').size()
arrival_counts.head()


/var/folders/8t/19klq0xd5w7cf39d3893ts0w0000gn/T/ipykernel_38055/253566319.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  arrival_counts = df.groupby('cluster').size()


cluster
Weekday - 0:00 - 4:00          0
Weekday - 4:00 - 8:00      15203
Weekday - 8:00 - 12:00     20785
Weekday - 12:00 - 16:00    21850
Weekday - 16:00 - 20:00    20992
dtype: int64

In [115]:
results = arrival_counts.to_frame(name='count')

results = results.merge(df[['cluster', 'cluster_hours']].drop_duplicates(), on='cluster', how='left')
results


,cluster,count,cluster_hours
0,Weekday - 0:00 - 4:00,0,NaN
1,Weekday - 4:00 - 8:00,15203,364.0
2,Weekday - 8:00 - 12:00,20785,364.0
3,Weekday - 12:00 - 16:00,21850,364.0
4,Weekday - 16:00 - 20:00,20992,364.0
5,Weekday - 20:00 - 0:00,1546,364.0
6,Weekend - 0:00 - 4:00,0,NaN
7,Weekend - 4:00 - 8:00,5973,152.0
8,Weekend - 8:00 - 12:00,7472,152.0
9,Weekend - 12:00 - 16:00,9494,152.0


In [116]:
results['avg_arrival_rate'] = results['count'] / results['cluster_hours'] / 60 # per minute
results


,cluster,count,cluster_hours,avg_arrival_rate
0,Weekday - 0:00 - 4:00,0,NaN,NaN
1,Weekday - 4:00 - 8:00,15203,364.0,0.696108
2,Weekday - 8:00 - 12:00,20785,364.0,0.951694
3,Weekday - 12:00 - 16:00,21850,364.0,1.000458
4,Weekday - 16:00 - 20:00,20992,364.0,0.961172
5,Weekday - 20:00 - 0:00,1546,364.0,0.070788
6,Weekend - 0:00 - 4:00,0,NaN,NaN
7,Weekend - 4:00 - 8:00,5973,152.0,0.654934
8,Weekend - 8:00 - 12:00,7472,152.0,0.819298
9,Weekend - 12:00 - 16:00,9494,152.0,1.041009


In [103]:
average_C0 = df.groupby('cluster')['C0 - S2'].mean().reset_index(name='avg_C0')
average_C0

/var/folders/8t/19klq0xd5w7cf39d3893ts0w0000gn/T/ipykernel_38055/441160710.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_C0 = df.groupby('cluster')['C0 - S2'].mean().reset_index(name='avg_C0')


,cluster,avg_C0
0,Weekday - 0:00 - 4:00,NaN
1,Weekday - 4:00 - 8:00,1.830626
2,Weekday - 8:00 - 12:00,1.264037
3,Weekday - 12:00 - 16:00,1.235103
4,Weekday - 16:00 - 20:00,1.372237
5,Weekday - 20:00 - 0:00,1.021992
6,Weekend - 0:00 - 4:00,NaN
7,Weekend - 4:00 - 8:00,1.776996
8,Weekend - 8:00 - 12:00,1.154042
9,Weekend - 12:00 - 16:00,1.203076


In [104]:
distribution_C0 = pd.crosstab(df['cluster'], df['C0 - S2'], normalize='index') * 100
distribution_C0.columns = [f'{col}_total_servers' for col in distribution_C0.columns]
distribution_C0


,0_total_servers,1_total_servers,2_total_servers,3_total_servers
cluster,,,,
Weekday - 4:00 - 8:00,0.000000,20.384135,76.169177,3.446688
Weekday - 8:00 - 12:00,0.000000,74.577821,24.440702,0.981477
Weekday - 12:00 - 16:00,0.000000,77.066362,22.356979,0.576659
Weekday - 16:00 - 20:00,0.000000,63.933880,34.908537,1.157584
Weekday - 20:00 - 0:00,0.064683,97.671410,2.263907,0.000000
Weekend - 4:00 - 8:00,0.000000,24.962330,72.375691,2.661979
Weekend - 8:00 - 12:00,0.000000,84.595824,15.404176,0.000000
Weekend - 12:00 - 16:00,0.000000,79.692437,20.307563,0.000000
Weekend - 16:00 - 20:00,0.000000,57.371104,38.698294,3.930602


In [105]:
results = average_C0.merge(distribution_C0, on='cluster', how='left')
results

,cluster,avg_C0,0_total_servers,1_total_servers,2_total_servers,3_total_servers
0,Weekday - 0:00 - 4:00,NaN,NaN,NaN,NaN,NaN
1,Weekday - 4:00 - 8:00,1.830626,0.000000,20.384135,76.169177,3.446688
2,Weekday - 8:00 - 12:00,1.264037,0.000000,74.577821,24.440702,0.981477
3,Weekday - 12:00 - 16:00,1.235103,0.000000,77.066362,22.356979,0.576659
4,Weekday - 16:00 - 20:00,1.372237,0.000000,63.933880,34.908537,1.157584
5,Weekday - 20:00 - 0:00,1.021992,0.064683,97.671410,2.263907,0.000000
6,Weekend - 0:00 - 4:00,NaN,NaN,NaN,NaN,NaN
7,Weekend - 4:00 - 8:00,1.776996,0.000000,24.962330,72.375691,2.661979
8,Weekend - 8:00 - 12:00,1.154042,0.000000,84.595824,15.404176,0.000000
9,Weekend - 12:00 - 16:00,1.203076,0.000000,79.692437,20.307563,0.000000


In [ ]:
# idk if different days makes sense anymore, I looked at the video again and he said the significant trends were based on weekday and weekend

# def imputation_cluster(timestamp):

#     day_name = timestamp.strftime('%A')  # day of week
#     hour = timestamp.hour
    
#     if 0 <= hour < 4:
#         time_cluster = "0:00 - 4:00"
#     elif 4 <= hour < 8:
#         time_cluster = "4:00 - 8:00"
#     elif 8 <= hour < 12:
#         time_cluster = "8:00 - 12:00"
#     elif 12 <= hour < 16:
#         time_cluster = "12:00 - 16:00"
#     elif 16 <= hour < 20:
#         time_cluster = "16:00 - 20:00"
#     else:
#         time_cluster = "20:00 - 0:00"
    

#     return f"{day_name} - {time_cluster}"

# df['imputation_cluster'] = df['S2'].apply(imputation_cluster)


In [ ]:
# imp_avg_wait = df.groupby('imputation_cluster')['Wait_Time'].mean()

# df['wait_time_imp'] = df['Wait_Time'].fillna(df['imputation_cluster'].map(imp_avg_wait))
# df['wait_time_imp'].hasnans


False

In [106]:
df['wait_time_imp']

57071     3.000000
56736     6.000000
56735     9.000000
91991     4.366075
91992    15.000000
           ...    
15664     3.000000
15665     3.000000
15666     3.000000
15661     1.000000
46802     2.000000
Name: wait_time_imp, Length: 114143, dtype: float64

In [107]:
#df['wait_time_imp'] = df['wait_time_imp'] / pd.Timedelta(minutes=1) # makes integer

thresholds = [5, 10, 15, 20, 25, 30]

results = {}

for cluster, group in df.groupby('cluster'):
    results[cluster] = {f"<= {threshold} mins": (group['wait_time_imp'] <= threshold).mean() * 100
                        for threshold in thresholds}

percentage_table_by_cluster = pd.DataFrame(results).transpose()
percentage_table_by_cluster.index.name = 'cluster'
percentage_table_by_cluster.reset_index(inplace=True)
percentage_table_by_cluster


/var/folders/8t/19klq0xd5w7cf39d3893ts0w0000gn/T/ipykernel_38055/1331756779.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for cluster, group in df.groupby('cluster'):


,cluster,<= 5 mins,<= 10 mins,<= 15 mins,<= 20 mins,<= 25 mins,<= 30 mins
0,Weekday - 0:00 - 4:00,NaN,NaN,NaN,NaN,NaN,NaN
1,Weekday - 4:00 - 8:00,45.306847,73.189502,88.028679,95.816615,98.618694,99.447477
2,Weekday - 8:00 - 12:00,60.250180,82.496993,92.090450,96.680298,98.349771,99.494828
3,Weekday - 12:00 - 16:00,70.274600,90.736842,97.052632,99.093822,99.510297,99.766590
4,Weekday - 16:00 - 20:00,77.948742,94.636052,98.589939,99.556974,99.857088,99.938072
5,Weekday - 20:00 - 0:00,93.208279,99.353169,99.935317,100.000000,100.000000,100.000000
6,Weekend - 0:00 - 4:00,NaN,NaN,NaN,NaN,NaN,NaN
7,Weekend - 4:00 - 8:00,56.437301,86.053909,97.170601,99.447514,99.933032,99.933032
8,Weekend - 8:00 - 12:00,68.388651,90.390792,96.654176,98.808887,99.571734,99.933084
9,Weekend - 12:00 - 16:00,70.644618,91.752686,98.325258,99.768275,99.957868,99.957868


In [110]:
cluster_metrics = df.groupby('cluster').apply(
    lambda group: pd.Series({
        'death_rate': len(group) / (group['cluster_hours'].min()),
        'avg_wait': group['wait_time_imp'].mean()
    })
).reset_index()

cluster_metrics['arrival_rate'] = (cluster_metrics['avg_wait'] * ((cluster_metrics['death_rate'])**2)) / (1 + cluster_metrics['death_rate'] * cluster_metrics['avg_wait'])

cluster_metrics['estimated_service_rate'] = (
    (cluster_metrics['avg_wait'] * cluster_metrics['arrival_rate'] + 
     np.sqrt((cluster_metrics['avg_wait'] * cluster_metrics['arrival_rate'])**2 + 
             4 * cluster_metrics['avg_wait'] * cluster_metrics['arrival_rate'])) 
    / (2 * cluster_metrics['avg_wait'])
)

cluster_metrics['traffic_intensity'] = cluster_metrics['arrival_rate'] / cluster_metrics['death_rate']

qos_results = []
for _, row in cluster_metrics.iterrows():
    cluster = row['cluster']
    arrival_rate = row['arrival_rate']
    estimated_service_rate = row['estimated_service_rate']
    
    qos_estimates = {}
    for x in thresholds:
        if estimated_service_rate > arrival_rate:
            qos = 1 - ((arrival_rate / estimated_service_rate) * np.exp(-(estimated_service_rate - arrival_rate) * x))
            qos_estimates[f"<= {x} mins"] = f"{qos:.1%}"
        else:
            qos_estimates[f"<= {x} mins"] = "N/A"
    
    result = {
        "Cluster": cluster,
        "Est Serv Rate": round(row['estimated_service_rate'], 3),
        "Est ρ": round(row['traffic_intensity'], 3),
        **qos_estimates
    }
    qos_results.append(result)

df_qos = pd.DataFrame(qos_results)
df_qos


/var/folders/8t/19klq0xd5w7cf39d3893ts0w0000gn/T/ipykernel_38055/1818786427.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cluster_metrics = df.groupby('cluster').apply(
/var/folders/8t/19klq0xd5w7cf39d3893ts0w0000gn/T/ipykernel_38055/1818786427.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cluster_metrics = df.groupby('cluster').apply(


,Cluster,Est Serv Rate,Est ρ,<= 5 mins,<= 10 mins,<= 15 mins,<= 20 mins,<= 25 mins,<= 30 mins
0,Weekday - 0:00 - 4:00,NaN,NaN,N/A,N/A,N/A,N/A,N/A,N/A
1,Weekday - 4:00 - 8:00,41.766,0.997,46.7%,71.5%,84.7%,91.8%,95.6%,97.7%
2,Weekday - 8:00 - 12:00,57.102,0.997,54.3%,79.0%,90.4%,95.6%,98.0%,99.1%
3,Weekday - 12:00 - 16:00,60.027,0.997,63.4%,86.6%,95.1%,98.2%,99.3%,99.8%
4,Weekday - 16:00 - 20:00,57.670,0.996,69.8%,90.9%,97.2%,99.2%,99.7%,99.9%
5,Weekday - 20:00 - 0:00,4.247,0.911,86.3%,97.9%,99.7%,100.0%,100.0%,100.0%
6,Weekend - 0:00 - 4:00,NaN,NaN,N/A,N/A,N/A,N/A,N/A,N/A
7,Weekend - 4:00 - 8:00,39.296,0.996,56.4%,80.9%,91.7%,96.3%,98.4%,99.3%
8,Weekend - 8:00 - 12:00,49.158,0.996,62.4%,85.8%,94.6%,98.0%,99.2%,99.7%
9,Weekend - 12:00 - 16:00,62.461,0.997,64.5%,87.4%,95.5%,98.4%,99.4%,99.8%


In [111]:
cluster_metrics

,cluster,death_rate,avg_wait,arrival_rate,estimated_service_rate,traffic_intensity
0,Weekday - 0:00 - 4:00,NaN,NaN,NaN,NaN,NaN
1,Weekday - 4:00 - 8:00,41.766484,7.969442,41.641380,41.766484,0.997005
2,Weekday - 8:00 - 12:00,57.101648,6.391859,56.945627,57.101648,0.997268
3,Weekday - 12:00 - 16:00,60.027473,4.975662,59.827165,60.027473,0.996663
4,Weekday - 16:00 - 20:00,57.670330,4.171182,57.431582,57.670330,0.995860
5,Weekday - 20:00 - 0:00,4.247253,2.401267,3.867993,4.247253,0.910705
6,Weekend - 0:00 - 4:00,NaN,NaN,NaN,NaN,NaN
7,Weekend - 4:00 - 8:00,39.296053,6.024855,39.130772,39.296053,0.995794
8,Weekend - 8:00 - 12:00,49.157895,5.109997,48.962976,49.157895,0.996035
9,Weekend - 12:00 - 16:00,62.460526,4.823921,62.253912,62.460526,0.996692


In [112]:
df_qos.to_csv("data-p3/new_mm1_qos_results.csv",index=False)

In [117]:
df_reg = average_C0
df_reg['avg_arrival_rate'] = results['avg_arrival_rate']
df_reg['est_service_rate'] = df_qos['Est Serv Rate'] / 60 # made seconds so easier to compare w arrival rate
df_reg['arrival_rate_per_server'] = df_reg['avg_arrival_rate'] / df_reg['avg_C0']
df_reg['service_rate_per_server'] = df_reg['est_service_rate'] / df_reg['avg_C0']
df_reg.head()


,cluster,avg_C0,avg_arrival_rate,est_service_rate,arrival_rate_per_server,service_rate_per_server
0,Weekday - 0:00 - 4:00,NaN,NaN,NaN,NaN,NaN
1,Weekday - 4:00 - 8:00,1.830626,0.696108,0.696100,0.380257,0.380253
2,Weekday - 8:00 - 12:00,1.264037,0.951694,0.951700,0.752901,0.752905
3,Weekday - 12:00 - 16:00,1.235103,1.000458,1.000450,0.810020,0.810013
4,Weekday - 16:00 - 20:00,1.372237,0.961172,0.961167,0.700442,0.700438


## appendix

### checking missing dates

In [93]:
df_check_S2 = pd.DataFrame(pd.to_datetime(df['S2']).dt.date)
df

# Specify the time period
start_date = '2028-08-01'
end_date = '2028-12-31'

# Generate the complete range of dates
full_date_range = pd.date_range(start=start_date, end=end_date,freq='D').date

# Find missing dates
present_dates = df_check_S2['S2'].unique()
missing_dates = [date for date in full_date_range if date not in present_dates]

print("Missing Dates:")
print(missing_dates)


Missing Dates:
[datetime.date(2028, 8, 2), datetime.date(2028, 8, 4), datetime.date(2028, 8, 7), datetime.date(2028, 8, 9), datetime.date(2028, 8, 11), datetime.date(2028, 8, 12), datetime.date(2028, 8, 13), datetime.date(2028, 8, 14), datetime.date(2028, 8, 15), datetime.date(2028, 8, 16), datetime.date(2028, 8, 17), datetime.date(2028, 8, 18), datetime.date(2028, 8, 19), datetime.date(2028, 8, 20), datetime.date(2028, 8, 21), datetime.date(2028, 8, 22), datetime.date(2028, 8, 23), datetime.date(2028, 8, 24), datetime.date(2028, 8, 25), datetime.date(2028, 8, 26), datetime.date(2028, 8, 27), datetime.date(2028, 8, 28), datetime.date(2028, 8, 29), datetime.date(2028, 8, 30)]


In [94]:
for i in missing_dates:
    print(i)

2028-08-02
2028-08-04
2028-08-07
2028-08-09
2028-08-11
2028-08-12
2028-08-13
2028-08-14
2028-08-15
2028-08-16
2028-08-17
2028-08-18
2028-08-19
2028-08-20
2028-08-21
2028-08-22
2028-08-23
2028-08-24
2028-08-25
2028-08-26
2028-08-27
2028-08-28
2028-08-29
2028-08-30


In [96]:
missing_info = pd.DataFrame({'Missing_Date': pd.to_datetime(missing_dates)})
missing_info['Day_Type'] = missing_info['Missing_Date'].dt.dayofweek.apply(lambda x: 'Weekday' if x < 5 else 'Weekend')

# Calculate the total number of weekdays and weekends
totals = missing_info['Day_Type'].value_counts()

print("Missing Dates with Classification:")
print(missing_info)
print("\nTotals:")
print(totals)

Missing Dates with Classification:
   Missing_Date Day_Type
0    2028-08-02  Weekday
1    2028-08-04  Weekday
2    2028-08-07  Weekday
3    2028-08-09  Weekday
4    2028-08-11  Weekday
5    2028-08-12  Weekend
6    2028-08-13  Weekend
7    2028-08-14  Weekday
8    2028-08-15  Weekday
9    2028-08-16  Weekday
10   2028-08-17  Weekday
11   2028-08-18  Weekday
12   2028-08-19  Weekend
13   2028-08-20  Weekend
14   2028-08-21  Weekday
15   2028-08-22  Weekday
16   2028-08-23  Weekday
17   2028-08-24  Weekday
18   2028-08-25  Weekday
19   2028-08-26  Weekend
20   2028-08-27  Weekend
21   2028-08-28  Weekday
22   2028-08-29  Weekday
23   2028-08-30  Weekday

Totals:
Day_Type
Weekday    18
Weekend     6
Name: count, dtype: int64
